In [26]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from dask.distributed import progress

In [27]:
SCARF_CONFIG = {
    'queue': 'scarf',
    'cores': 24,
    'memory': '128GB',
}

# create a cluster
cluster = SLURMCluster(**SCARF_CONFIG)

# connect a client and scale to 10 workers
client = Client(cluster)
cluster.scale(20)

/home/vol01/scarf1041/programs/miniconda3/envs/py39/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41176 instead
  warnings.warn(


In [28]:
cluster

In [29]:
import time
import socket
import os

def do_work():
    time.sleep(0.1)
    process_id = os.getpid()
    hostname = socket.gethostname()
    return f'{hostname}_{process_id}'

In [36]:
# run locally
do_work()

'ui4.scarf.rl.ac.uk_1674336'

In [37]:
delayed_work = dask.delayed(do_work)()

In [38]:
delayed_work

Delayed('do_work-ac24284a-65f6-4b19-8c5a-ec8aa29d1a4d')

In [39]:
# work is done on the cluster when the compute() method is called
delayed_work.compute()

'cn804.scarf.rl.ac.uk_115708'

In [40]:
from datetime import datetime

In [41]:
# local
results = []

t0 = datetime.now()
for i in range(100):
    result = do_work()
    results.append(result)
t1 = datetime.now()

time_taken = (t1 - t0).total_seconds()

In [42]:
print(f'time taken (1 cpu): {time_taken:.2f} s')

time taken (1 cpu): 10.02 s


In [43]:
# distributed
results = []

t0 = datetime.now()
for i in range(100):
    result = dask.delayed(do_work)()
    results.append(result)

results = dask.compute(*results)
t1 = datetime.now()

time_taken = (t1 - t0).total_seconds()
number_of_unique_hosts = len(set([result.split('_')[0] for result in results]))
number_of_unique_processes = len(set(results))

In [44]:
print(f'time taken (distributed): {time_taken:.2f} s')
print(f'number of unique hosts: {number_of_unique_hosts}')
print(f'number of unique processes: {number_of_unique_processes}')

time taken (distributed): 0.31 s
number of unique hosts: 4
number of unique processes: 24
